# HW_Webscraping
- 무신사 페이지 - 랭킹 - 아우터부분
- 한 페이지 당 90개, 10페이지 가져오기
- 가격 변수와 다른 변수와의 상관성과 각 변수들의 분포, 브랜드의 독점성 등을 확인하고자 선택

> URL : https://search.musinsa.com/ranking/best?&mainCategory=002&page=1
- mainCategory=002 : outer
- &page=1 : 1페이지

> 가져올 변수
- goods : 상품이름
- brand : 브랜드
- price : 가격
- review_count : 리뷰 수

In [36]:
import warnings
warnings.filterwarnings("ignore") 

import requests
from bs4 import BeautifulSoup
import re

Musinsa_URL = "https://search.musinsa.com/ranking/best?&mainCategory=002"
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

result = requests.get(Musinsa_URL, headers = headers)

def extract_outer_ranking(html):
    goods_info = html.find('div',{'class':'article_info'})
    try:
        goods = goods_info.find('p', {'class':'list_info'}).get_text().strip().split('\n')[-1].strip()
    except:
        goods = None
    try:
        # brand
        brand = goods_info.find('p', {'class':'item_title'}).get_text().strip()
    except:
        brand = None
    try:
        # price
        price = goods_info.find('p',{'class':'price'}).get_text().strip().split('원')[0]
    except:
        price = None
    try:
        # review_count
        review_count = goods_info.find('span',{'class':'count'}).get_text().strip()
    except:
        review_count = None
    
    return {
        'goods' : goods,
        'brand' : brand,
        'price' : price,
        'review_count' : review_count
    }
    
def extract_outer_rankings():
    outers = []
    for page in range(10):
        print("Scrapping Musinsa outer Ranking Page : {}".format(page+1))
        result = requests.get("{0}&page={1}".format(Musinsa_URL,page+1))
        soup = BeautifulSoup(result.text, "html.parser")
        results = soup.find_all("li", {"class":"li_box"})
        for result in results:
            outer = extract_outer_ranking(result)
            outers.append(outer)
    return outers

def get_outers():
    musinsa_outers = extract_outer_rankings()
    print('\nMusinsa outer Ranking Page Scraping complete')
    
    return musinsa_outers

In [37]:
musinsa_outers = get_outers()

Scrapping Musinsa outer Ranking Page : 1
Scrapping Musinsa outer Ranking Page : 2
Scrapping Musinsa outer Ranking Page : 3
Scrapping Musinsa outer Ranking Page : 4
Scrapping Musinsa outer Ranking Page : 5
Scrapping Musinsa outer Ranking Page : 6
Scrapping Musinsa outer Ranking Page : 7
Scrapping Musinsa outer Ranking Page : 8
Scrapping Musinsa outer Ranking Page : 9
Scrapping Musinsa outer Ranking Page : 10

Musinsa outer Ranking Page Scraping complete


In [38]:
import pandas as pd

data = pd.DataFrame(musinsa_outers)

In [42]:
data

,goods,brand,price,review_count
0,캐시미어 더블 롱 코트 BLACK,인사일런스,"289,000","8,526"
1,[이동휘 착용][3M 신슐레이트]오리지널 M-1965 피쉬테일 파카_Original...,라퍼지스토어,"138,000","6,446"
2,MTR 오버핏 더블 롱 코트 (블랙),쿠어,"334,000","5,255"
3,[PRAUDEN] 유틸리티 덕다운 푸파 숏패딩_라이트그레이,코드그라피,"124,000","1,870"
4,울리치 트렌치 맥코트_Black,라퍼지 포 우먼,"163,000","2,444"
...,...,...,...,...
895,울리치 오버핏 블루종 점퍼_Black,라퍼지스토어,"108,000",39
896,비전3 봄버 다운 자켓 YELLOW,코닥,"419,000",6
897,[3M 신슐레이트]쉬링 피셔맨 패딩_Gray,라퍼지스토어,"114,000",20
898,ECO LEATHER SINGLE BREASTED BLAZER BROWN,프렌다,"169,000",10


In [41]:
# 겹처서 가져온 데이터 없음
data.drop_duplicates()

,goods,brand,price,review_count
0,캐시미어 더블 롱 코트 BLACK,인사일런스,"289,000","8,526"
1,[이동휘 착용][3M 신슐레이트]오리지널 M-1965 피쉬테일 파카_Original...,라퍼지스토어,"138,000","6,446"
2,MTR 오버핏 더블 롱 코트 (블랙),쿠어,"334,000","5,255"
3,[PRAUDEN] 유틸리티 덕다운 푸파 숏패딩_라이트그레이,코드그라피,"124,000","1,870"
4,울리치 트렌치 맥코트_Black,라퍼지 포 우먼,"163,000","2,444"
...,...,...,...,...
895,울리치 오버핏 블루종 점퍼_Black,라퍼지스토어,"108,000",39
896,비전3 봄버 다운 자켓 YELLOW,코닥,"419,000",6
897,[3M 신슐레이트]쉬링 피셔맨 패딩_Gray,라퍼지스토어,"114,000",20
898,ECO LEATHER SINGLE BREASTED BLAZER BROWN,프렌다,"169,000",10


In [43]:
data.to_excel('Musinsa_Data.xlsx')